Imports

In [12]:
import csv
import pandas as pd
from random import shuffle
import tensorflow as tf
from langdetect import detect
import google_transs

In [13]:
with open('train_set.csv', 'rb') as f:
    train_set = pd.read_csv(f)

with open('test_set.csv', 'rb') as f:
    test_set = pd.read_csv(f)

Text Cleaning

In [33]:
text = "Gamarjveba bichebo sakartvelo iamakebs tkvenit"

def linkRemoval(text):
    if 'http' in text:
        t = ""
        s = -1
        for i in range(0,len(text)):
            if(s!=-1 and text[i]==' '):
                s=-1  
            if(i+3 < len(text)):
                if(text[i]+text[i+1]+text[i+2]+text[i+3] == 'http'):
                    s=i
            if(s==-1):
                t+=text[i]
        text = t
    return text

def transLang(text):
    try:
        lang = detect(text)
        translator = google_transs.google_translator()
        if(lang=='en' or lang=='ru' or lang=='pl'):
            text = translator.translate(text, lang_tgt='ka')
    except:
        return text
    return text
    

convertor = {
    'a' : 'ა',
    'b' : 'ბ',
    'g' : 'გ',
    'd' : 'დ',
    'e' : 'ე',
    'v' : 'ვ',
    'z' : 'ზ',
    'i' : 'ი',
    'k' : 'კ',
    'l' : 'ლ',
    'm' : 'მ',
    'n' : 'ნ',
    'o' : 'ო',
    'p' : 'პ',
    'r' : 'რ',
    's' : 'ს',
    't' : 'ტ',
    'u' : 'უ',
    'f' : 'ფ',
    'q' : 'კ',
    'y' : 'ყ',
    'sh': 'შ',
    'ch': 'ჩ',
    'ts': 'ც',
    'c' : 'ც',
    'dz': 'ძ',
    'w' : 'ჭ',
    'x' : 'ხ',
    'j' : 'ჯ',
    'h' : 'ჰ',
}


def isLatin(text):
    latinWordCounter = 0
    geoWordCounter = 0
    for i in text:
        if((i>'a' and i<'z') or (i>'A' and i<'Z')):
            latinWordCounter += 1
        if(i>'ა' and i<'ჰ'):
            geoWordCounter += 1
    return (latinWordCounter > geoWordCounter)

def removeDuplicateLetters(text):
    i=0
    c=0
    r=""
    while(i<len(text)):
        if(i+1<len(text)):
            if(text[i]==text[i+1]):
                c+=1
            else:
                c=0
        else:
            c=0
        if(c<2):
            r+=text[i]
        i+=1
    return r

def convertionLang(text):
    text = linkRemoval(text)
    text = transLang(text)
    islat = isLatin(text)
    if(islat == False):
        text = cleanGeoText(text)
    else:
        text = text.lower()
        text = cleanLatinText(text)
    
    text = removeDuplicateLetters(text)
    return text

def cleanGeoText(text):
    res = ''
    for i in text:
        if(i=='თ'):
            i='ტ'
        elif(i=='წ'):
            i='ჭ'
        elif(i=='ჟ'):
            i='ჯ'
        elif(i=='ღ'):
            i='გ'
        elif(i=='ქ'):
            i='კ'
        res += i
    return res

def cleanLatinText(text):
    res = ''
    i=0
    
    while(i<len(text)):
        c = text[i]
        if(i+1 < len(text)):
            n = text[i+1]
            if(convertor.get(c+n) != None):
                res+=convertor.get(c+n)
                i+=1
            elif(convertor.get(c) != None):
                res+=convertor.get(c)
            else:
                res+=c    
        else:
            if(convertor.get(c) != None):
                res+=convertor.get(c)
            else:
                res+=c
        i+=1
    return res
newText = convertionLang(text)
print(newText)

გამარჯვება ბიჩებო საკარტველო იამაკებს ტკვენიტ
